# Benchmark for the emission test tracer

The emission test tracer is transported in ICON-AES. The emission test tracer tendency is updated using Python code and the established bridges given in the field attribute `echam_ttr_config%icon_ml_bridge`. 

- `fortran`: Pure Fortran
- `cffi`
- `pipes`
- `mpi`

The benchmark is done on a one-month atm amip simulation where no output is written beyond the basic necessities.

## Read the times directly from log files

Keep the human out of the loop to avoid errors.

Cycle all the log files, check if they finished successfully, and retrieve the wall time

We go the lazy route and just load the full file, worst case the jupyterhub crashes 

In [ ]:
import pandas as pd
import itertools
import os
from datetime import datetime
import numpy as np

In [ ]:
if os.path.exists('./benchmark.csv'):
    df = pd.read_csv('./benchmark.csv')
else:
    df = pd.DataFrame(columns=['count', 'resolution', 'bridge', 'nodes', 'procs_pernode', 't_start', 't_end', 'logfile', 'is_logfile', 'is_finished_ok'])

In [ ]:
%%time

exp_dir = '/work/ka1176/caroline/jobs/hereon_iconml/one_month/mpi_procs_64'

var_nodes = [1, 2, 4]
var_procs_pernode = [128, 64, 32, 16]
var_counts = [1, 2]
var_bridges = ['fortran', 'cffi', 'pipes', 'mpi']
var_resolutions = ['R02B04']

refresh_frame = True # refresh all entries --> False for speed up

i=0
for nodes, procs_pernode, count, bridge, resolution in itertools.product(var_nodes, var_procs_pernode, var_counts, var_bridges, var_resolutions):
    
    # check if entry already exists in dataframe
    entry = df[df.eval(f"count=={count} & nodes=={nodes} & procs_pernode=={procs_pernode}")]
    entry = entry.loc[entry['bridge'] == bridge] # stupid string comparison does not work in eval
    entry = entry.loc[entry['resolution'] == resolution]
    
    assert len(entry) <= 1
    
    refresh_ix = None
    
    if len(entry)==1 and entry['is_finished_ok'].values[0] is True:
        if refresh_frame:
            refresh_ix = entry.index
            print(f'Entry exists, refreshing index {refresh_ix}')
        else:
            continue
    
    logfile = f'LOG_exp.iconml_month_{bridge}_{nodes}-{procs_pernode}_{count}.run'
    is_logfile = os.path.exists(os.path.join(exp_dir, logfile))
    
    # none-initialize the values that are parsed from logfiles
    t_start = None
    t_end = None
    is_finished_ok = False
    
    # check if the run finished OK
    if is_logfile:
        with open(os.path.join(exp_dir, logfile)) as f:
            ll = f.readlines()
            
            is_finished_ok = [ 'Script run successfully:  OK' in lll for lll in ll ]
            is_finished_ok = np.sum(np.asarray(is_finished_ok)) == 2 # occurs twice in log script
            
            # read the start and end time
            if is_finished_ok:
                ix = np.where(np.asarray(ll) == '+ date\n')[0]
                assert len(ix) == 2
                t_start = ll[ix[0]+1].strip()
                t_end   = ll[ix[1]+1].strip()
    
    new_entry = pd.DataFrame(dict(count=count,
                               resolution=resolution,
                               bridge=bridge,
                               nodes=nodes,
                               procs_pernode=procs_pernode,
                               t_start=t_start,
                               t_end=t_end,
                               logfile=logfile,
                               is_logfile=is_logfile,
                               is_finished_ok=is_finished_ok), index=[i])
    
    if refresh_ix is None:
        df = pd.concat([df, new_entry], axis=0)
    else:
        df.iloc[refresh_ix] = new_entry
    i += 1

In [ ]:
df

In [ ]:
df.loc[df['is_finished_ok']]

In [ ]:
df.to_csv('./benchmark.csv')

## Process

- Calculate wall time
- Calculate total number of MPI cores

In [ ]:
#df = df.loc[df['is_finished_ok']]
df.info()

In [ ]:
df['MPI_processes'] = df['nodes'] * df['procs_pernode']

In [ ]:
def time_stamp_to_seconds(s, tsformat='%a %b %d %H:%M:%S %Z %Y', to_sec=True):
    '''
    Converts a time stamp from an ICON log script to UNIX seconds
    
    Parameters:
    
    s - Time stamp string
    tsformat - Format see https://strftime.org/
    to_sec - Return as Epoch seconds (default: True)
    
    Returns:
    Parsed time stamp in requested format
    '''
    
    if s == 'None': # typecast as str
        return None
        
    x = datetime.strptime(s, tsformat)
    
    if to_sec:
        return int(x.strftime('%s'))
    return x

In [ ]:
df_sec = df[['t_start', 't_end']].astype(str).applymap(time_stamp_to_seconds)
df_sec

In [ ]:
df['delta_t'] = df_sec['t_end'] - df_sec['t_start']

## Plot

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

sns.set_style('whitegrid')
sns.set_context('talk')

### For different numbers of mpi_procs

In [ ]:
for procs in np.sort(pd.unique(df.MPI_processes)):
    print(procs)
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))

    sns.barplot(data=df.loc[df['MPI_processes'] == procs], x='nodes', y='delta_t', hue='bridge', ax=ax)

    ax.set_title(f'One month R02B04 with {procs} MPI processes')
    ax.set_ylabel('Wall time (seconds)')

    fig.tight_layout()
    plt.show()

### Bridges compared

In [ ]:
fig = plt.figure(figsize=(8, 5))

sns.lineplot(data=df, x='MPI_processes', y='delta_t', hue='bridge', legend='brief', marker='o')
ax=plt.gca()
ax.set(xscale="log", yscale="log")
ax.set_xlabel('MPI processes')
ax.set_ylabel('Wall time (seconds)')

xtix = np.sort(pd.unique(df.MPI_processes)).astype(int)
ax.set_xticks(xtix)
ax.set_xticklabels(xtix)
plt.show()


Compare the runtime when bridge is included with the original Fortran runtime

In [ ]:
for procs in np.sort(pd.unique(df.MPI_processes)):
    print('-'*40)
    print(' MPI processes: ', procs)
    
    tmp = df.loc[(df['MPI_processes']==procs) & (df['bridge'] != 'mpi')].groupby('bridge')['delta_t'].mean()
    
    if tmp.isnull().all():
        continue

    tmp =  100 * (tmp - tmp.fortran) / tmp.fortran
    
    print('Relative increase (%) compared to FORTRAN runtime\n', tmp)

### Variability